In [1]:
import chipwhisperer as cw

In [2]:
try:
    if not scope.connectStatus:
        scope.con()
except NameError:
    scope = cw.scope()

try:
    if SS_VER == "SS_VER_2_1":
        target_type = cw.targets.SimpleSerial2
    elif SS_VER == "SS_VER_2_0":
        raise OSError("SS_VER_2_0 is deprecated. Use SS_VER_2_1")
    else:
        target_type = cw.targets.SimpleSerial
except:
    SS_VER="SS_VER_1_1"
    target_type = cw.targets.SimpleSerial

try:
    target = cw.target(scope, target_type)
except:
    print("INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.")
    print("INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.")
    scope = cw.scope()
    target = cw.target(scope, target_type)


print("INFO: Found ChipWhisperer😍")
scope.default_setup()

INFO: Found ChipWhisperer😍
scope.gain.mode                          changed from low                       to high                     
scope.gain.gain                          changed from 0                         to 30                       
scope.gain.db                            changed from 5.5                       to 24.8359375               
scope.adc.state                          changed from True                      to False                    
scope.adc.basic_mode                     changed from low                       to rising_edge              
scope.adc.samples                        changed from 24400                     to 5000                     
scope.adc.trig_count                     changed from 941173                    to 14621488                 
scope.clock.adc_src                      changed from clkgen_x1                 to clkgen_x4                
scope.clock.adc_freq                     changed from 0                         to 29538459          

In [3]:
# clocks:
print('Set the CW305 J16 switch to 1 so that Ibex is clocked from HS2. Note this is different from what most other CW305 notebooks require.')

scope.clock.clkgen_src = 'system'
scope.clock.clkgen_freq = 100e6
if scope._is_husky:
    scope.clock.adc_mul = 1
else:
    scope.clock.adc_src = 'clkgen_x1'

target.baud = 115200

Set the CW305 J16 switch to 1 so that Ibex is clocked from HS2. Note this is different from what most other CW305 notebooks require.


In [4]:
# Ibex requires external JTAG programming
prog = None

# But let's program the FPGA bitfile:
from chipwhisperer.hardware.firmware.open_fw import getsome_generator
getsome = getsome_generator("cw305")
bsdata = getsome(f"lowrisc_ibex_demo_system_512KRAM.bit")
cw305 = cw.target(None, cw.targets.CW305, bsfile=None, force=False)
status = cw305.fpga.FPGAProgram(bsdata, exceptOnDoneFailure=False, prog_speed=10e6)

if status:
    print("✅ FPGA programmed. Next you need to program the firmware using the load_demo_system.sh command from the Ibex repository.")
else:
    print(status)
    print("❌ FPGA Done pin failed to go high")

(ChipWhisperer Target WARNING|File CW305.py:508) No FPGA Bitstream file specified.


✅ FPGA programmed. Next you need to program the firmware using the load_demo_system.sh command from the Ibex repository.


In [5]:
def reset_target(scope):
    raise IOError("Default Ibex build does not have Python-driven reset - use the R1 button on the CW305, or the S1 button on the CW313")


## Upload the software

In [6]:
import subprocess
import time

In [7]:
def primitive_reset(verbose=True, demo_prog="basic-passwdcheck"):
    ibex_base_path = ""
    cmd = f"../../../ibex-demo-system/util/load_demo_system.sh run ../../../ibex-demo-system/sw/c/build/demo/{demo_prog}/{demo_prog}"
    # FIXME: Add code to confirm file existence
    try:
        # Execute the command and capture output
        result = subprocess.run(cmd, shell=True, check=True, 
                               stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                               text=True)
        
        # Print the output
        if verbose:
            print("Command executed successfully!")
            print("Output:")
    
            for line in result.stderr.splitlines():
                print(line)
            
        return result
    
    except subprocess.CalledProcessError as e:
        print(f"Error executing command. Return code: {e.returncode}")
        print(f"Error message: {e.stderr}")
        return e
    
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")
        return e

In [8]:
start = time.time()
res = primitive_reset(demo_prog="simpleserial-aes")
end = time.time()
print(f"Time elapsed: {end-start}")

Command executed successfully!
Output:
Open On-Chip Debugger 0.12.0+dev-04253-gfa7e2351c (2025-03-06-13:47)
Licensed under GNU GPL v2
For bug reports, read
	http://openocd.org/doc/doxygen/bugs.html
force hard breakpoints
Info : ftdi: if you experience problems at higher adapter clocks, try the command "ftdi tdo_sample_edge falling"
Info : clock speed 10000 kHz
Info : JTAG tap: riscv.cpu tap/device found: 0x13631093 (mfg: 0x049 (Xilinx), part: 0x3631, ver: 0x1)
Info : [riscv.cpu] datacount=2 progbufsize=8
Info : [riscv.cpu] Examined RISC-V core
Info : [riscv.cpu]  XLEN=32, misa=0x40101104
[riscv.cpu] Target successfully examined.
Info : [riscv.cpu] Examination succeed
Info : [riscv.cpu] starting gdb server on 3333
Info : Listening on port 3333 for gdb connections
riscv.cpu halted due to debug-request.
10924 bytes written at address 0x00100000
downloaded 10924 bytes in 0.043311s (246.311 KiB/s)
verified 10924 bytes in 0.042594s (250.457 KiB/s)
Doing reset
Info : JTAG tap: riscv.cpu tap/d

In [9]:
def cap_pass_trace(pass_guess):
    primitive_reset(verbose = False) # Slow
    num_char = target.in_waiting()
    while num_char > 0:
        target.read(num_char, 10)
        time.sleep(0.01)
        num_char = target.in_waiting()

    scope.arm()
    target.write(pass_guess)
    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')

    trace = scope.get_last_trace()
    return trace

In [10]:
scope.adc.samples = 6000

In [11]:
trace_h = cap_pass_trace("h\n")
trace_0 = cap_pass_trace("0\n")
print(trace_h)
cw.plot(trace_h)*cw.plot(trace_0)

[ 0.39355469  0.16601562  0.36035156 ... -0.04980469  0.25488281
  0.09960938]


:Overlay
   .Curve.I  :Curve   [x]   (y)
   .Curve.II :Curve   [x]   (y)

In [12]:
#key = bytearray([0x72,0x23,0xd5,0x31,0x01,0xcd,0x3a,0x77,0x83,0xde,0x29,0x24,0x7f,0xff,0x19,0xa6])
#ktp = cw.ktp.Basic()
#trace_array = []
#textin_array = []
#proj = cw.create_project("CPA_no_noise")
#target.simpleserial_write('k', key)
#N = 100
#for i in range(N):
#    unused, text = ktp.next()

#    trace = cw.capture_trace(scope, target, text, key)
#    if not trace:
#        continue

#proj.traces.append(trace)

#leak_model = cwa.leakage_models.sbox_output
#attack = cwa.cpa(proj_noise, leak_model)
#cb = cwa.get_jupyter_callback(attack)
#results_noise = attack.run(cb, 25)